<h2>PROJET SANTE PUBLIQUE - Nettoyage des données<h2>

Nous importons des les packages dont nous avons besoin. A noter l'utilisation du package missingno permettant la visualisation des données manquantes

**Approche générale:**

Nous allons construiure une application de 'Health tracking' qui propose de scanner les aliments quotidiens qu'un utilisateur consomme quotidiennement. Notre application va donner une note à chacun des aliments. Ensuite, à la fin de la journée, l'utilisateur recevra une note concernant globale pour la journée, concernant sa nutrition. 

Nous allons faire ici l'étape de nettoyage des données. 

Ayant identifié les features cohérents avec notre application, nous avons identifiés les colonnes utiles de notre csv, et allons extraire ces colonnes. Notre travail de nettoyage se fera à partir de cette extraction.

In [ ]:
import os
import pandas as pd
import numpy as np
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import KNNImputer
pd.options.mode.chained_assignment = None  # default='warn'

Nous changons de working directory pour simplifier import et export

Nous lancons la lecture de notre fichier via Pandas. A noter que nous allons extraire directement les features qui nous interessent, 'used_features' ci-dessous. Cela aura l'avantage aussi d'être moins consommateur en mémoire, donc moins de temps de calcul.

Les données initiales sont sépararés par une tabulation.

In [ ]:
filepath = '../input/openfoodfactscsv/openfoodfacts.csv'
used_features = ['product_name','brands','code','pnns_groups_1','nutriscore_grade','energy-kcal_100g','proteins_100g','carbohydrates_100g','sugars_100g','fat_100g','saturated-fat_100g','fiber_100g','sodium_100g','nutrition-score-fr_100g']
data = pd.read_csv(filepath, sep='\t', usecols=used_features)

In [ ]:
pd.options.display.float_format = "{:.1f}".format

In [ ]:
data.shape

In [ ]:
data.describe()

In [ ]:
data.head()

Nous voyons ici que notre jeu de données filtré contient beaucoup de NaN, soulignant un travail de nettoyage d'abord nécessaire de ce point de vue.

Dans le graphique ci-dessous, nous visualisons la complétude de nos features, et remarquons ainsi qu'ils sont plutôt non-NaN, excepté quatres features: additives_n, nutriscore_grade, fibre_100g, nutriscore-fr_100g

In [ ]:
msno.bar(data, figsize=(10,5));

Nous créons une copie de notre jeu de données pour ne pas altérer le jeu initial

In [ ]:
data_copy=data.copy(deep=True)

Nous savons que certaines features de notre base de données ont des caractérisques particulières. Beaucoup sont des valeurs comprises entre 0 et 100. Nous donnons le nom de "non_negative_capped_data" à ces features. Nous en profitons pour discuter les données numériques de manière générale

In [ ]:
non_negative_capped_data = ['fat_100g','saturated-fat_100g','carbohydrates_100g','sugars_100g','fiber_100g','proteins_100g','sodium_100g']
numeric_data = ['energy-kcal_100g','fat_100g','saturated-fat_100g','carbohydrates_100g','sugars_100g','fiber_100g','proteins_100g','sodium_100g','nutrition-score-fr_100g']

Ci-dessous nous traitons les valeurs abérrantes de la facon suivante: 
- si un point de "non_negative_capped_data" est négatif, nous méttons sa valeur à 0 (valeur minimale légale)
- si un point de "non_negative_capped_data" est supérieur à 100, nous méttons sa valeur à 100 (valeur maximale légale)
- pour energy-kcal, nous limitons la valeur maximale à 3000kcal

In [ ]:
for col in non_negative_capped_data:
    data_copy[col][data_copy[col] < 0] = 0
    data_copy[col][data_copy[col] > 100] = 100
data_copy['energy-kcal_100g'][data_copy['energy-kcal_100g'] > 1500] = 1500

In [ ]:
data_copy[['sugars_100g','fiber_100g','proteins_100g','sodium_100g']].describe()

In [ ]:
data_copy.shape

In [ ]:
data_copy.head()

Nous pouvons maintenant enlever les lignes de faible valeur ajoutée: ce sont les lignes dont les valeurs sont uniquement des NaNs.

In [ ]:
new_data = data_copy.dropna(how='all',subset=numeric_data)
new_data = new_data.reset_index(drop=True)

In [ ]:
new_data.tail()

In [ ]:
new_data.shape

In [ ]:
range(len(new_data.index))

In [ ]:
ind = []
for value in range(len(new_data.index)):
   if new_data['product_name'].loc[value] == 'Eau':
      ind.append(value) 
new_data.loc[ind].head(20)

In [ ]:
new_data = new_data.reset_index(drop=True)

Nous allons procéder à la préparation de l'utilisation de KNNImputer. Nous copions les données numériques dans un dataframe, et copions les valeurs numériques dans full_num

In [ ]:
secondary_data=new_data[:][['code','product_name','pnns_groups_1','nutriscore_grade']]
secondary_data = secondary_data.reset_index(drop=True)

In [ ]:
full_num = new_data[numeric_data]

In [ ]:
random_sample_full_num=full_num.sample(20000) 
random_sample_full_num

Nous avons fait le choix de fitter notre modèle sur 20,000 lignes choisis de manière aléatoire pour alléger le temps de calcul dans le processus de transformation. Ainsi, nous obtenons data_Knned après 2h30 de temps de calcul

In [ ]:
imputer = KNNImputer(n_neighbors=3)
imputer.fit(random_sample_full_num)

In [ ]:
data_Knned = imputer.transform(full_num)

In [ ]:
df = pd.DataFrame(data_Knned,columns = numeric_data)

In [ ]:
df.head()

Nous fusionnons les valeurs numériques aves le dataframe secondary_data

In [ ]:
final = pd.concat([secondary_data, df], axis=1)

In [ ]:
final

In [ ]:
def grader_food(x):
        if x<=-1:
            return 'a'
        elif (x>-1)&(x<=2):
            return 'b'
        elif (x>2)&(x<=10):
            return 'c'
        elif (x>10)&(x<=18):
            return 'd'
        else:
            return 'e'

def grader_beverages(x):
        if (x<=0):
            return 'a'
        elif (x>0)&(x<=1):
            return 'b'
        elif (x>1)&(x<=5):
            return 'c'
        elif (x>5)&(x<=9):
            return 'd'
        else:
            return 'e'

In [ ]:
for ind in range(len(final.index)):
    if final['nutriscore_grade'].loc[ind] is np.nan:
        if final['pnns_groups_1'].loc[ind] == 'Beverages':
            final['nutriscore_grade'].loc[ind] = grader_beverages(final['nutrition-score-fr_100g'].loc[ind])
        else:
            final['nutriscore_grade'].loc[ind] = grader_food(final['nutrition-score-fr_100g'].loc[ind])

In [ ]:
final

In [ ]:
final.to_csv('../output', index=False)